In [5]:
# Read/create the text data
# Let’s create a list of strings and assign it to a variable. 
# Maybe a tweet sample:
tweet_sample= "How to take control of your #debt https://personal.vanguard.com/us/insights/saving-investing/debt-management. #Best advice for #family #financial #success (@PrepareToWin)"


In [6]:
#import nltk
#nltk.download('wordnet')

# Execute the below function to process the tweet:
def processRow(row):
 import re
 import nltk
 from textblob import TextBlob
 from nltk.corpus import stopwords
 from nltk.stem import PorterStemmer
 from textblob import Word
 from nltk.util import ngrams
 from nltk.tokenize import word_tokenize
 
 tweet = row

#Lower case
 tweet.lower()

#Removes unicode strings like "\u002c"  -> ,(comma)
 tweet = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', tweet)
    
# Removes non-ascii characters. note : \x00 to \x7f is 00 to 255
# non-ascii characters like copyrigth symbol, trademark symbol
 tweet = re.sub(r'[^\x00-\x7f]',r'',tweet)
               
#convert any url to URL
 tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
               
#Convert any @Username to "AT_USER"
 tweet = re.sub('@[^\s]+','AT_USER',tweet)

#Replace multiple white spaces with one white space
 tweet = re.sub('[\s]+', ' ', tweet)
#Replace multiple break-line i.e enter-key,  with one white space
 tweet = re.sub('[\n]+', ' ', tweet)

# Removes hastag in front of a word
# or simply said Replace #word with word
 tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

#Replace non-word chars with a single white space
 tweet = re.sub(r'[^\w]', ' ', tweet)
  
# Remove smiley face symbols
 tweet = tweet.replace(':)','')
 tweet = tweet.replace(':(','')
# below we have removed all possible emoticons / smiley faces

#Removes emoticons from text
 tweet = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', tweet)

#remove numbers
 tweet = ''.join([i for i in tweet if not i.isdigit()])

#remove multiple exclamation  -> this is optional
 tweet = re.sub(r"(\!)\1+", ' ', tweet)

#remove multiple question marks -> this is optional
 tweet = re.sub(r"(\?)\1+", ' ', tweet)

#remove multistop -> this is optional
 tweet = re.sub(r"(\.)\1+", ' ', tweet)

#trim -> this is optional, as this would have been removed 
# by the above [^\w] step
 tweet = tweet.strip('\'"')

# making lemma of the cleaned word
 from textblob import Word
 tweet =" ".join([Word(word).lemmatize() for word in tweet.split()])

 row = tweet
 return row
               
# call the function with your data
processRow(tweet_sample)

'How to take control of your debt URL Best advice for family financial success AT_USER'

**Note** : The above text pre-processing (*`cleaning`*) function totally depends on type / nature of data. One has to accordingly code his function. Above is just an example to clean tweet data. Depending on the kind of cleaning you plan on the data the above function can go from simple to complex coding.

In [1]:
# load the modules / libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation

In [ ]:
# load the Training Data set
train = pd.read_csv("quora_train_set.csv", index_col='id')[:1000]
train.head()

In [ ]:
# Check for any null values
print(train.isnull().sum())

In [ ]:
# Preview some of the pairs of questions
a = 0 
for i in range(a,a+10):
    print(train.question1[i])
    print(train.question2[i])
    print()

In [ ]:
# we have defined our own list of Stop words
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']

In [ ]:
def text_to_wordlist(text, remove_stop_words=True, stem_words=False):
    # Clean the text, 
    # with the option to remove stop_words 
    # and to stem words.

    # Clean the text in your own way.  This gives better results. 
    # So instead of using only regex, we have identified the 
    # short forms people type & would use in english
    # and replaced them.
    # This list could go even bigger.
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    # Please note , we could have also generated lemma. 
    # from textblob import Word
    # text = " ".join([Word(word).lemmatize() for word in text.split()])

       
    # Return a list of words
    return(text)

In [ ]:
# calling the above user defined function
def process_questions(question_list, questions, 
                      question_list_name, 
                      dataframe):
    '''transform questions and display progress'''
    for question in questions:
        question_list.append(text_to_wordlist(question))
        if len(question_list) % 100 == 0:
            progress = len(question_list)/len(dataframe) * 100
            print("{} is {}% complete.".format(question_list_name, round(progress, 1)))

In [ ]:
# Creating a empty list to hold the cleaned Question
train_question1 = []

# calling the above user defn fn.
process_questions(train_question1, train.question1, 
                  'train_question1', train)

In [ ]:
# Creating a empty list to hold the cleaned Question
train_question2 = []

# calling the above user defn fn.
process_questions(train_question2, train.question2, 
                  'train_question2', train)

In [ ]:
# Preview some transformed pairs of questions
a = 0 
for i in range(a,a+10):
    print(train_question1[i])
    print(train_question2[i])
    print()